# Script para generar la solución del Primer Benchmark de la Competencia

## Si no presentaste aún tu primera solución, tenes la oportunidad de hacerlo en pocos Clicks!

**Hola! **  
  
Este Script es un Ejemplo de Procesamiento de los Datos, Modelado y Generación de una Solución.

Agregamos una pequeña explicación de lo que se hace en cada paso para ayudar a los que están comenzando ahora


### Importamos las librerías que vamos a utilizar

In [47]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, ParameterGrid
import re

In [ ]:
#from sklearn.model_selection import KFold, ParameterGrid

### Lectura de las Bases

Observamos los datos que tenemos disponibles en https://www.kaggle.com/c/interbank20/data

Vamos a trabajar ahora con todas las bases disponibles

In [2]:
rcc_train = pd.read_csv("/kaggle/input/interbank20/rcc_train.csv")
se_train = pd.read_csv("/kaggle/input/interbank20/se_train.csv", index_col="key_value")
censo_train = pd.read_csv("/kaggle/input/interbank20/censo_train.csv", index_col="key_value")
y_train = pd.read_csv("/kaggle/input/interbank20/y_train.csv", index_col="key_value").target

rcc_test= pd.read_csv("/kaggle/input/interbank20/rcc_test.csv")
se_test= pd.read_csv("/kaggle/input/interbank20/se_test.csv", index_col="key_value")
censo_test= pd.read_csv("/kaggle/input/interbank20/censo_test.csv", index_col="key_value")

### Vamos a trabajar ahora con la base de **RCC**:
* Discretizamos los días de atraso para poder manipularla mejor
* Hacemos tablas cruzadas sobre key_value y cada variable de interés, utilizando distintas funciones de agregación sobre el saldo del producto
* condicion son los dias de mora

In [3]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins)
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins)

In [5]:
def makeCt(df, c, aggfunc=sum):
    try:
        ct = pd.crosstab(df.key_value, df[c].fillna("N/A"), values=df.saldo, aggfunc=aggfunc)
    except:
        ct = pd.crosstab(df.key_value, df[c], values=df.saldo, aggfunc=aggfunc)
    ct.columns = [f"{c}_{aggfunc.__name__}_{v}" for v in ct.columns]
    return ct

In [6]:
train = []
test = []
#aggfuncs = [len, sum]
aggfuncs = [sum]
for c in rcc_train.drop(["codmes", "key_value", "saldo", "tipo_credito", "cod_instit_financiera", "condicion", "RIESGO_DIRECTO", "COD_CLASIFICACION_DEUDOR"], axis=1):
    print("haciendo", c)
    train.extend([makeCt(rcc_train, c, aggfunc) for aggfunc in aggfuncs])
    test.extend([makeCt(rcc_test, c, aggfunc) for aggfunc in aggfuncs])

haciendo PRODUCTO


In [7]:
train = pd.concat(train, axis=1)
test = pd.concat(test, axis=1)

In [8]:
train.head

<bound method NDFrame.head of            PRODUCTO_sum_0  PRODUCTO_sum_1  PRODUCTO_sum_2  PRODUCTO_sum_3  \
key_value                                                                   
0                0.462743       -0.723839       -0.458168       -0.501221   
1                     NaN       -0.218150             NaN             NaN   
2                     NaN       -0.145561             NaN             NaN   
3                     NaN       -0.143971             NaN             NaN   
4               -0.816867       -0.362155       -0.233343       -0.262546   
...                   ...             ...             ...             ...   
358482          -0.050498       -0.072814             NaN             NaN   
358483          -0.061673       -0.072803             NaN             NaN   
358484          -0.059344       -0.072702             NaN             NaN   
358485          -0.058336             NaN             NaN             NaN   
358486                NaN       -0.072724     

### Incorporamos la Información adicional existente en las tablas socio económicas y del censo. Es un simple join porque ambas tienen key_value únicos
#### Por el momento no incorporamos la información tributaria porque requiere un tratamiento más complejo que queda para futuras revisiones

In [9]:
train = train.join(censo_train).join(se_train)
test = test.join(censo_test).join(se_test)

### Por la naturaleza de las variables creadas, nos aseguramos que solo se utilicen variables existentes en ambos conjuntos de datos (train y test)

In [10]:
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols]
test = test[keep_cols]
len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))

(0, 0)

In [11]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

### Entrenamiento del Modelo

Para entrenar nuestro modelo vamos a usar LightGBM

In [12]:
folds = [train.index[t] for t, v in KFold(5).split(train)]

In [50]:
params = ParameterGrid({"num_leaves": [10 , 20], "max_depth": [4 , 6, 8]})

In [52]:
for param in params:
    name = ";".join([f"{k}_{v}" for k, v in param.items()])
    print(name)

max_depth_4;num_leaves_10
max_depth_4;num_leaves_20
max_depth_6;num_leaves_10
max_depth_6;num_leaves_20
max_depth_8;num_leaves_10
max_depth_8;num_leaves_20


In [ ]:
best_score = 0
best_probs = []
for param in params:
    test_probs = []
    train_probs = []
    p = ";".join([f"{k}_{v}" for k, v in param.items()]) 
    print("*"*10, p, "*"*10)
    for i, idx in enumerate(folds):
        Xt = train.loc[idx]
        yt = y_train.loc[Xt.index]

        Xv = train.drop(Xt.index)
        yv = y_train.loc[Xv.index]

        learner = LGBMClassifier(n_estimators=1000, **param)
        learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                    eval_set=[(Xt, yt), (Xv, yv)], verbose=False)
        test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
        train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
        fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

    test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
    train_probs = pd.concat(train_probs)
    score = roc_auc_score(y_train, train_probs.loc[y_train.index])
    print("*" * 21)
    print(f"roc auc estimado para {p}: {score}")
    if score > best_score:
        best_probs = test_probs

********** max_depth_4;num_leaves_10 **********
*********************
roc auc estimado para max_depth_4;num_leaves_10: 0.7356000618710998
********** max_depth_4;num_leaves_20 **********
*********************
roc auc estimado para max_depth_4;num_leaves_20: 0.736410033579695
********** max_depth_6;num_leaves_10 **********
*********************
roc auc estimado para max_depth_6;num_leaves_10: 0.7362601169073424
********** max_depth_6;num_leaves_20 **********
*********************
roc auc estimado para max_depth_6;num_leaves_20: 0.7373226002234491
********** max_depth_8;num_leaves_10 **********
*********************
roc auc estimado para max_depth_8;num_leaves_10: 0.7364427168739108
********** max_depth_8;num_leaves_20 **********


### Guardado del modelo para hacer la presentación

Finalmente creamos el archivo CSV que podemos subir como nuestra Solución a la competencia

Empezá con este archivo y luego podes seguir mejorándolo a ver si subís en posiciones!

In [ ]:
best_probs.name = "target"
best_probs.to_csv("benchmark1.csv")